In [90]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [91]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [92]:
train_dataset = datasets.MNIST(root='Data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='Data', train=False, transform=transforms.ToTensor(), download=True)

In [93]:
train_dataset[0]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [94]:
type(train_dataset[0][0])

torch.Tensor

In [95]:
train_dataset[0][0].size()   # [#channels, height, width]

torch.Size([1, 28, 28])

In [96]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

In [97]:
train_loader

In [98]:
for i in train_loader:
    print(len(i))      
    print(len(i[0]))  
    print(len([i][0][0]))
    break

2
64
64


### नमूना

In [103]:
class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 50)    # hidden layer 1
        self.fc2 = nn.Linear(50, 10)     # o/p layer
        
    def forward(self, x):
        x = self.fc1(x)     # passing features through hidden layer 1
        x = F.relu(x)       # Applying relu on the features passed through hidden layer 1
        x = self.fc2(x)     # passing features throgh hidden layer 2
        x = F.softmax(x,dim=1)
        return x
    
model = ANN().to(device)
print(model)

ANN(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


### प्रशिक्षण

In [111]:
num_epochs = 1
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(num_epochs):
    for batch_data in train_loader:
        X, y = batch_data    # X(batch of features) & y(batch of corresponding labels)
        optimizer.zero_grad()
        X = X.to(device)
        y = y.to(device)
        X = X.view(X.shape[0],-1)    
        
        y_ = model(X)
        loss = loss_function(y_, y)
        loss.backward()
        optimizer.step()

### परीक्षण

In [112]:
correct = 0
total = 0

for batch_data in test_loader:
    X, y = batch_data
    X = X.to(device)
    y = y.to(device)
    X = X.view(X.shape[0], -1)
    y_ = model(X)
    for idx, i in enumerate(y_):
        if torch.argmax(i) == y[idx]:correct += 1
        total += 1

### यथार्थता

In [113]:
correct/total

0.9578666666666666